<a href="https://colab.research.google.com/github/pamelabrg/mandelbrot-generator/blob/main/mandelbrot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [102]:
%%writefile mandelbrot.c

#include <stdio.h>
#include <stdlib.h>

void mandelbrot(int width, int height, int max_iter, double center_x, double center_y, double zoom, const char* filename) {
    FILE* fp = fopen(filename, "wb");
    if (!fp) {
        perror("fopen");
        exit(EXIT_FAILURE);
    }

    double real_range = 4.0 / zoom;
    double imag_range = 4.0 / zoom;
    double real_min = center_x - (real_range / 2);
    double real_max = center_x + (real_range / 2);
    double imag_min = center_y - (imag_range / 2);
    double imag_max = center_y + (imag_range / 2);

    fwrite(&width, sizeof(int), 1, fp);
    fwrite(&height, sizeof(int), 1, fp);
    fwrite(&max_iter, sizeof(int), 1, fp);
    fwrite(&real_min, sizeof(double), 1, fp);
    fwrite(&real_max, sizeof(double), 1, fp);
    fwrite(&imag_min, sizeof(double), 1, fp);
    fwrite(&imag_max, sizeof(double), 1, fp);

    unsigned char* image = (unsigned char*)malloc(width * height);
    if (!image) {
        perror("malloc");
        exit(EXIT_FAILURE);
    }

    for (int y = 0; y < height; ++y) {
        for (int x = 0; x < width; ++x) {
            double cx = real_min + (x / (double)width) * (real_max - real_min);
            double cy = imag_min + (y / (double)height) * (imag_max - imag_min);
            double zx = 0.0, zy = 0.0;
            int i;
            for (i = 0; i < max_iter; ++i) {
                double xtemp = zx * zx - zy * zy + cx;
                zy = 2.0 * zx * zy + cy;
                zx = xtemp;
                if (zx * zx + zy * zy > 4.0) break;
            }
            image[y * width + x] = (unsigned char)(255.0 * i / max_iter);
        }
    }

    fwrite(image, sizeof(unsigned char), width * height, fp);
    fclose(fp);
    free(image);
}

int main(int argc, char* argv[]) {
    if (argc != 8) {
        fprintf(stderr, "Usage: %s width height max_iter center_x center_y zoom output_file\n", argv[0]);
        return EXIT_FAILURE;
    }

    int width = atoi(argv[1]);
    int height = atoi(argv[2]);
    int max_iter = atoi(argv[3]);
    double center_x = atof(argv[4]);
    double center_y = atof(argv[5]);
    double zoom = atof(argv[6]);
    const char* output_file = argv[7];

    mandelbrot(width, height, max_iter, center_x, center_y, zoom, output_file);
    return EXIT_SUCCESS;
}


Overwriting mandelbrot.c


In [103]:
!gcc mandelbrot.c -o mandelbrot -lm

In [105]:
import subprocess
import numpy as np
from PIL import Image

def generate_fractal(width, height, max_iter, center_x, center_y, zoom, binary_file, png_file):
    subprocess.run([
        './mandelbrot',
        str(width), str(height), str(max_iter),
        str(center_x), str(center_y),
        str(zoom), binary_file
    ], check=True)

    with open(binary_file, 'rb') as f:
        width = np.fromfile(f, dtype=np.int32, count=1)[0]
        height = np.fromfile(f, dtype=np.int32, count=1)[0]
        max_iter = np.fromfile(f, dtype=np.int32, count=1)[0]
        real_min = np.fromfile(f, dtype=np.float64, count=1)[0]
        real_max = np.fromfile(f, dtype=np.float64, count=1)[0]
        imag_min = np.fromfile(f, dtype=np.float64, count=1)[0]
        imag_max = np.fromfile(f, dtype=np.float64, count=1)[0]

        image_data = np.fromfile(f, dtype=np.uint8).reshape((height, width))

    image = Image.fromarray(image_data, mode='L')
    image.save(png_file)

def main():
    width = 1080
    height = 1080
    max_iter = int(input("Número máximo de iterações: "))
    center_x = float(input("Coordenada real do centro: "))
    center_y = float(input("Coordenada imaginária do centro: "))
    zoom = float(input("Fator de zoom: "))
    binary_file = 'mandelbrot_data.bin'
    png_file = 'mandelbrot.png'

    generate_fractal(width, height, max_iter, center_x, center_y, zoom, binary_file, png_file)

if __name__ == '__main__':
    main()
1

Largura da imagem: 1080
Altura da imagem: 1080
Número máximo de iterações: 500
Coordenada real do centro: -0.861
Coordenada imaginária do centro: -0.63
Fator de zoom: 1.9157
